<a href="https://colab.research.google.com/github/bdemirc1/chemBERT/blob/main/ChemBERT_10K_density_hof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Nvidia Apex for dealing with 16-bit training-less memory but mixed precision training
# Apex alleviates these problems.
!git clone https://github.com/NVIDIA/apex
%cd apex
!pwd


Cloning into 'apex'...
remote: Enumerating objects: 11442, done.
remote: Counting objects: 100% (3510/3510), done.
remote: Compressing objects: 100% (432/432), done.
remote: Total 11442 (delta 3218), reused 3156 (delta 3076), pack-reused 7932
Receiving objects: 100% (11442/11442), 15.40 MiB | 14.53 MiB/s, done.
Resolving deltas: 100% (8047/8047), done.
/content/apex
/content/apex


In [2]:
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="bdist_wheel" --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
Processing /content/apex
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.7 MB/s eta 0:00:00
    Getting requirements to build wheel: started
    Running command Getting requirements to build wheel
    running egg_info
    writing setuptools.egg-info/PKG-INFO
    writing dependency_links to setuptools.egg-info/dependency_links.txt
    writing entry points to setuptools.egg-info/entry_points.txt
    writing requirements to setuptools.egg-info/requires.txt
    writing top-level names to setuptools.egg-info/top_level.txt
    readin

In [3]:
# Required for simple transformer classification model - trained using f16 - true: 16 bit
from apex import amp

In [4]:
!pip install transformers
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.6 MB/s eta

In [6]:
!nvidia-smi

Wed Nov  8 16:49:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import os

import numpy as np
import pandas as pd


In [6]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

In [7]:
!pip install transformers
!pip install simpletransformers

In [8]:
!pip install wandb

In [9]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [10]:
!pwd
df_density = pd.read_csv("../10k_dft_density_data.csv")
df_density.dtypes
df_density.info

/content/apex


<bound method DataFrame.info of                                                   smiles   density
0                           CC(C)OC(=O)c1ccc(cc1)N(=O)=O  1.367633
1                 CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC  1.292618
2                              CC(C)(C)c1ccc(O)c(C=NO)c1  1.227993
3                      Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O  1.316755
4      CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...  1.406207
...                                                  ...       ...
10201         CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C  1.299822
10202                        CON=C(C)C(=O)C=CC=Cc1ccccc1  1.220360
10203                          CON=C(C)C(=O)C=CC1=CC=CN1  1.261450
10204      CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O  1.579706
10205  Cc1c(nc2CC(C)(C)CC(=O)c2c1c1ccc(cc1)N(=O)=O)c1...  1.333787

[10206 rows x 2 columns]>

In [11]:
new_column_names = {
    "smiles": "text",
    "density": "labels",
}
df_density.rename(columns=new_column_names, inplace=True)
df_density

,text,labels
0,CC(C)OC(=O)c1ccc(cc1)N(=O)=O,1.367633
1,CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC,1.292618
2,CC(C)(C)c1ccc(O)c(C=NO)c1,1.227993
3,Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O,1.316755
4,CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...,1.406207
...,...,...
10201,CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C,1.299822
10202,CON=C(C)C(=O)C=CC=Cc1ccccc1,1.220360
10203,CON=C(C)C(=O)C=CC1=CC=CN1,1.261450
10204,CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O,1.579706


In [12]:
df_hof = pd.read_csv("../10k_dft_hof_data.csv")
df_hof.dtypes
df_hof.info

<bound method DataFrame.info of                                                   smiles       hof_s
0                           CC(C)OC(=O)c1ccc(cc1)N(=O)=O -109.985346
1                 CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC -100.962611
2                              CC(C)(C)c1ccc(O)c(C=NO)c1  -55.199618
3                      Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O   34.675794
4      CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...   49.411500
...                                                  ...         ...
10201         CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C  -99.026131
10202                        CON=C(C)C(=O)C=CC=Cc1ccccc1  -12.290865
10203                          CON=C(C)C(=O)C=CC1=CC=CN1  -12.217791
10204      CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O   29.753252
10205  Cc1c(nc2CC(C)(C)CC(=O)c2c1c1ccc(cc1)N(=O)=O)c1...  -33.978039

[10206 rows x 2 columns]>

In [13]:
new_column_names = {
    "smiles": "text",
    "hof_s": "labels",
}
df_hof.rename(columns=new_column_names, inplace=True)
df_hof

,text,labels
0,CC(C)OC(=O)c1ccc(cc1)N(=O)=O,-109.985346
1,CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC,-100.962611
2,CC(C)(C)c1ccc(O)c(C=NO)c1,-55.199618
3,Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O,34.675794
4,CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...,49.411500
...,...,...
10201,CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C,-99.026131
10202,CON=C(C)C(=O)C=CC=Cc1ccccc1,-12.290865
10203,CON=C(C)C(=O)C=CC1=CC=CN1,-12.217791
10204,CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O,29.753252


In [14]:
from sklearn.model_selection import train_test_split

train_density_df, temp_density_df = train_test_split(df_density, test_size=0.3, random_state=42)
validation_density_df, test_density_df = train_test_split(temp_density_df, test_size=0.5, random_state=42)

train_hof_df, temp_hof_df = train_test_split(df_hof, test_size=0.3, random_state=42)
validation_hof_df, test_hof_df = train_test_split(temp_hof_df, test_size=0.5, random_state=42)

In [15]:
train_density_df

,text,labels
7539,CN1OC(C)(C)CC1(C#N)C1COC2(CCCCC2)O1,1.252277
8806,CC(=O)C1=CN(C=C(C1c1ccc(cc1)N(=O)=O)C(C)=O)c1c...,1.365263
9560,CC(C)C1=NC2=C(O1)c1ccccc1C(=O)C2=NOC(C)=O,1.391895
6425,CC1C2C(CON2C(C)(C)C)CN1CC=C,1.139091
8511,CN(C)CCN(Cc1c2ccccc2cc2ccccc12)c1ccc(cn1)N(=O)=O,1.319714
...,...,...
5734,CC(=O)OCC1OC(Oc2ccc(cc2N(=O)=O)N(=O)=O)C(OC(C)...,1.531870
5191,CC(C)(C)C(CN(=O)=O)C(=O)c1ccc2OCOc2c1,1.371337
5390,CC1(C)CC(=O)C2=C(C1)OC(=NO)C2c1cccnc1,1.344472
860,CON1C(=O)c2ccccc2C1=CC(=O)NC(C)C,1.312522


In [16]:
test_density_df.columns

Index(['text', 'labels'], dtype='object')

In [17]:
train_hof_df

,text,labels
7539,CN1OC(C)(C)CC1(C#N)C1COC2(CCCCC2)O1,-98.462167
8806,CC(=O)C1=CN(C=C(C1c1ccc(cc1)N(=O)=O)C(C)=O)c1c...,-47.801184
9560,CC(C)C1=NC2=C(O1)c1ccccc1C(=O)C2=NOC(C)=O,-90.737533
6425,CC1C2C(CON2C(C)(C)C)CN1CC=C,-13.755196
8511,CN(C)CCN(Cc1c2ccccc2cc2ccccc12)c1ccc(cn1)N(=O)=O,39.822818
...,...,...
5734,CC(=O)OCC1OC(Oc2ccc(cc2N(=O)=O)N(=O)=O)C(OC(C)...,-483.022332
5191,CC(C)(C)C(CN(=O)=O)C(=O)c1ccc2OCOc2c1,-132.532307
5390,CC1(C)CC(=O)C2=C(C1)OC(=NO)C2c1cccnc1,-51.932802
860,CON1C(=O)c2ccccc2C1=CC(=O)NC(C)C,-83.245164


In [19]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.evaluate_each_epoch = True
model_args.evaluate_during_training_verbose = True
model_args.no_save = True
model_args.num_train_epochs = 10
model_args.wandb_project = 'chemBert10k-density'

In [20]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [21]:
# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "seyonec/ChemBERTA_PubChem1M_shard00_155k",
    num_labels=1,
    args=model_args
)

# Train the model
model.train_model(train_density_df)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/ChemBERTA_PubChem1M_shard00_155k and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/7144 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

wandb: Currently logged in as: bdemirc1 (bingedu). Use `wandb login --relogin` to force relogin


Running Epoch 0 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

(8930, 0.014139905444327205)

In [22]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(validation_density_df)

  0%|          | 0/1531 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Convert text input data to list of strings
text_inputs = test_density_df['text'].tolist()

# Predict target values
predictions, model_outputs = model.predict(text_inputs)
print(predictions)

# Calculate MAE and MSE
mae = mean_absolute_error(test_density_df['labels'], predictions)
mse = mean_squared_error(test_density_df['labels'], predictions)
rmse = np.sqrt(mse)

# Print the results
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)

  0%|          | 0/1531 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]

[1.359375   1.37988281 1.65136719 ... 1.48046875 1.35253906 1.39648438]
Mean Absolute Error: 0.06128994939347295
Mean Squared Error: 0.004011667595251535
Root Mean Squared Error: 0.06333772647681266


In [25]:
# Hof data training
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.evaluate_each_epoch = True
model_args.evaluate_during_training_verbose = True
model_args.no_save = True
model_args.num_train_epochs = 10
model_args.wandb_project = 'chemBert-10k-hof'
model_args.overwrite_output_dir = True

In [26]:
!wandb login

wandb: Currently logged in as: bdemirc1 (bingedu). Use `wandb login --relogin` to force relogin


In [27]:
model = ClassificationModel(
    "roberta",
    "seyonec/ChemBERTA_PubChem1M_shard00_155k",
    num_labels=1,
    args=model_args
)

# Train the model
model.train_model(train_hof_df)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/ChemBERTA_PubChem1M_shard00_155k and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/7144 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Training loss,█▅▄▅▂▁▃▂▄▃▁▃▂▁▂▁▂▁▂▁▂▂▁▂▁▃▁▁▂▁▁▁▂▁▁▁▁▁▁▁
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,▂▅████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
Training loss,0.00328
global_step,8900
lr,0.0


Running Epoch 0 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

(8930, 7144.5668763097165)

In [28]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(validation_hof_df)

  0%|          | 0/1531 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

In [29]:
text_inputs = test_hof_df['text'].tolist()

predictions, model_outputs = model.predict(text_inputs)
print(predictions)

# Calculate MAE and MSE
mae = mean_absolute_error(test_hof_df['labels'], predictions)
mse = mean_squared_error(test_hof_df['labels'], predictions)
rmse = np.sqrt(mse)

# Print the results
print("HoF Mean Absolute Error:", mae)
print("HoF Mean Squared Error:", mse)
print("HoF Root Mean Squared Error:", rmse)

  0%|          | 0/1531 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]

[-117.125        40.75        -30.8125     ...   18.234375      5.43359375
   21.671875  ]
HoF Mean Absolute Error: 29.290208701879546
HoF Mean Squared Error: 4182.499241209469
HoF Root Mean Squared Error: 64.67224475158929
